In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd

In [ ]:
!unzip your_file.zip

unzip:  cannot find or open your_file.zip, your_file.zip.zip or your_file.zip.ZIP.


In [ ]:
!gsutil cp gs://your-bucket-name/your_file.csv .

ServiceException: 401 Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist).


In [ ]:
data=pd.read_csv("/content/validation.csv.zip")
data

,article,abstract
0,the interest in anchoring phenomena and phenom...,we study the phase behavior of a nematic liqui...
1,"galaxy clusters , as the largest peaks in the ...",determining the scaling relations between gala...
2,quantum correlations between components of a s...,we show how to control spatial quantum correla...
3,methanol masers are often found in star - form...,class i methanol masers are believed to be pr...
4,interdisciplinary research has recently gained...,"nowadays , scientific challenges usually requi..."
...,...,...
6431,"in the gauge theory , only the gauge invariant...",we give a formal proof that the space - time a...
6432,the stability of thin films on substrates has ...,we consider here the effects of inertia on the...
6433,density - functional theory ( dft ) @xcite is...,we introduce an orbital - optimized double - h...
6434,laser - compton scattering is an attractive me...,we have been developing optical resonant cavit...


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')
nltk.download('punkt_tab')

stop_words = set(stopwords.words('english'))
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
import re

def clean_arxiv_article(text):

    text = re.split(r'\b(references|appendix|acknowledgements)\b', text, flags=re.IGNORECASE)[0]
    text = re.sub(r'\\begin{.*?}|\\end{.*?}', '', text)
    text = re.sub(r'\$.*?\$', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word not in stop_words]

    return " ".join(tokens)

data['clean_article']=data['article'].apply(clean_arxiv_article)

In [ ]:
def clean_text(text):
    words = text.split()
    cleaned_words = [words[0]] if words else []
    for word in words[1:]:
        if word != cleaned_words[-1]:
            cleaned_words.append(word)

    # حذف الكلمات الغريبة (مثلاً: ccccc, xxxxx, aaaa)
    final_words = []
    for word in cleaned_words:
        if not re.fullmatch(r'([a-zA-Z])\1{2,}', word):
            final_words.append(word)

    return ' '.join(final_words)

data['final_clean_article']=data['clean_article'].apply(clean_text)

In [ ]:
data.head(2)

,article,abstract,clean_article,final_clean_article
0,the interest in anchoring phenomena and phenom...,we study the phase behavior of a nematic liqui...,interest anchoring phenomena phenomena confine...,interest anchoring phenomena confined nematic ...
1,"galaxy clusters , as the largest peaks in the ...",determining the scaling relations between gala...,galaxy clusters largest peaks cosmic density f...,galaxy clusters largest peaks cosmic density f...


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

In [ ]:
train_data["length"] = train_data["final_clean_article"].apply(lambda x: len(x.split()))
print(train_data["length"].describe())

count     5792.000000
mean      1883.374309
std       1495.530034
min          6.000000
25%        795.000000
50%       1622.500000
75%       2571.250000
max      18161.000000
Name: length, dtype: float64


In [ ]:
def chunk_text(text, max_words=512):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_words):
        chunk = ' '.join(words[i:i+max_words])
        chunks.append(chunk)
    return chunks

def prepare_chunked_dataset(data_df, max_words=512):
    chunked_data = []

    for _, row in data_df.iterrows():
        chunks = chunk_text(row["final_clean_article"], max_words=max_words)
        for chunk in chunks:
            chunked_data.append({
                "input_text": "abstract: " + chunk,
                "target_text": row["abstract"]
            })

    return pd.DataFrame(chunked_data)

In [ ]:
chunked_df_train = prepare_chunked_dataset(train_data, max_words=1024)
chunked_df_test = prepare_chunked_dataset(test_data, max_words=1024)

In [ ]:
print(chunked_df_train)

                                              input_text  \
0      abstract: characterizing large scale distribut...   
1      abstract: core globular clusters thought star ...   
2      abstract: lot interest spin ladders decade mai...   
3      abstract: interested dissecting squares smalle...   
4      abstract: prime dissection integer lengths gra...   
...                                                  ...   
13714  abstract: clusters members derive delivery tre...   
13715  abstract: style node discovery technique basic...   
13716  abstract: lcc overlay whereas stabilizes rough...   
13717  abstract: agn feedback critical element state ...   
13718  abstract: main theorem thm main theorem shows ...   

                                             target_text  
0      we present the first analysis of three - dimen...  
1      we investigate the tidal interactions of a red...  
2      we analyze a possibility of quantum criticalit...  
3      dissections of a square into smaller

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
dataset_train = Dataset.from_pandas(chunked_df_train[['input_text', 'target_text']].rename(columns={
    "input_text": "text",
    "target_text": "summary"}))
dataset_test = Dataset.from_pandas(chunked_df_test[['input_text', 'target_text']].rename(columns={
    "input_text": "text",
    "target_text": "summary"}))

In [ ]:
def preprocess(example):
    model_inputs = tokenizer(example['text'], max_length=1024, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example['summary'], max_length=250, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train = dataset_train.map(preprocess, batched=True)
tokenized_test = dataset_test.map(preprocess, batched=True)

Map:   0%|          | 0/13719 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1507 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
import torch
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_dir="./logs",
    logging_strategy="epoch",
    report_to="none",
)



In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipython-input-24-2075482251.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,3.049100,2.679535


Epoch,Training Loss,Validation Loss
1,3.049100,2.679535
2,2.882100,3.049235
3,3.450500,3.166498


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=10290, training_loss=3.1272313836248786, metrics={'train_runtime': 9465.1096, 'train_samples_per_second': 4.348, 'train_steps_per_second': 1.087, 'total_flos': 5.006879369588736e+16, 'train_loss': 3.1272313836248786, 'epoch': 3.0})

In [ ]:
trainer.save_model("best_t5_model")

In [ ]:
!zip -r best_t5_model.zip best_t5_model
from google.colab import files
files.download("best_t5_model.zip")

  adding: best_t5_model/ (stored 0%)
  adding: best_t5_model/training_args.bin (deflated 52%)
  adding: best_t5_model/model.safetensors (deflated 8%)
  adding: best_t5_model/spiece.model (deflated 48%)
  adding: best_t5_model/generation_config.json (deflated 27%)
  adding: best_t5_model/config.json (deflated 63%)
  adding: best_t5_model/tokenizer_config.json (deflated 94%)
  adding: best_t5_model/special_tokens_map.json (deflated 85%)
  adding: best_t5_model/added_tokens.json (deflated 83%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_path = "best_t5_model"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

In [ ]:
def chunk_text(text, tokenizer, max_length=512):
    tokens = tokenizer.encode(text, truncation=False)
    chunks = []
    for i in range(0, len(tokens), max_length):
        chunk = tokens[i:i + max_length]
        chunk_text = tokenizer.decode(chunk, skip_special_tokens=True)
        chunks.append(chunk_text)
    return chunks

In [ ]:
def summarize(text):
    text = clean_text(text)
    chunks = chunk_text(text, tokenizer)

    summaries = []
    for chunk in chunks:
        input_text = "summarize: " + chunk
        input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids,
                max_length=150,
                num_beams=4,
                early_stopping=True
            )

        summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        summaries.append(summary)

    return " ".join(summaries)

sample_text = """
large language models such as GPT-3 have demonstrated impressive capabilities in generating human-like text.
they are trained on massive datasets and can perform a variety of tasks without explicit supervision.
however, there remain concerns about their reliability, interpretability, and potential biases.
one proposed approach involves using smaller, fine-tuned models for specific tasks.
this approach can reduce computational costs and increase performance on domain-specific data.
"""
summary = summarize(sample_text)
print("Summary:\n", summary)

Summary:
 large language models such as GPT-3 have demonstrated impressive capabilities in generating human-like text . however, there remain concerns about their reliability, interpretability, and potential biases . one proposed approach involves using smaller, fine-tuned models for specific tasks . this approach can reduce computational costs and increase performance on domain-specific data .


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=fb8fa68fc3f9fbbd18e30f86378c93dc9a6b247a5c0301e880b0e939903b97ce
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import evaluate

rouge = evaluate.load("rouge")
reference = """large language models such as GPT-3 have demonstrated impressive capabilities in generating human-like text . they are trained on massive datasets and can perform a variety of tasks without explicit supervision . however, there remain concerns about their reliability, interpretability, and potential biases . one proposed approach involves using smaller, fine-tuned models for specific tasks . this approach can reduce computational costs and increase performance on domain-specific data ."""
generated = """large language models such as GPT-3 have demonstrated impressive capabilities in generating human-like text . however, there remain concerns about their reliability, interpretability, and potential biases . one proposed approach involves using smaller, fine-tuned models for specific tasks . this approach can reduce computational costs and increase performance on domain-specific data ."""

results = rouge.compute(predictions=[generated], references=[reference])
print(results)


{'rouge1': np.float64(0.8666666666666666), 'rouge2': np.float64(0.847457627118644), 'rougeL': np.float64(0.8666666666666666), 'rougeLsum': np.float64(0.8666666666666666)}
